In [1]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt

In [2]:
import yaml

In [14]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())


print(launch_command(conf))

export CUDA_VISIBLE_DEVICES=0
python -m vllm.entrypoints.openai.api_server --model hatakeyama-llm-team/Tanuki-8B-Instruct --max-model-len 4000 --port 8000 --gpu-memory-utilization 0.2 

export CUDA_VISIBLE_DEVICES=1
python -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-medium-128k-instruct --max-model-len 4000 --port 8001 --gpu-memory-utilization 0.4 

export CUDA_VISIBLE_DEVICES=2
python -m vllm.entrypoints.openai.api_server --model Rakuten/RakutenAI-7B-chat --max-model-len 4000 --port 8002 --gpu-memory-utilization 0.2 

export CUDA_VISIBLE_DEVICES=3,4
python -m vllm.entrypoints.openai.api_server --model karakuri-ai/karakuri-lm-8x7b-chat-v0.1 --max-model-len 4000 --port 8010 --tensor-parallel-size 2 




In [15]:
client_dict=get_client_dict(conf)
client_dict

{'hatakeyama-llm-team/Tanuki-8B-Instruct': {'client': <openai.OpenAI at 0x7fd33776c610>,
  'config': {'name': 'hatakeyama-llm-team/Tanuki-8B-Instruct',
   'PORT': 8000,
   'max-model-len': 4000,
   'gpu-memory-utilization': 0.2,
   'GPU_ID': 0,
   'template1': '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n',
   'template2': '\n\n### 応答:\n',
   'max_tokens': 2000}},
 'microsoft/Phi-3-medium-128k-instruct': {'client': <openai.OpenAI at 0x7fd33776f590>,
  'config': {'name': 'microsoft/Phi-3-medium-128k-instruct',
   'PORT': 8001,
   'max-model-len': 4000,
   'gpu-memory-utilization': 0.4,
   'GPU_ID': 1,
   'template1': '<|user|>\n',
   'template2': '<|end|>\n<|assistant|>',
   'max_tokens': 2000}},
 'Rakuten/RakutenAI-7B-chat': {'client': <openai.OpenAI at 0x7fd337771c50>,
  'config': {'name': 'Rakuten/RakutenAI-7B-chat',
   'PORT': 8002,
   'model': 'Rakuten/RakutenAI-7B-chat',
   'max-model-len': 4000,
   'gpu-memory-utilization': 0.2,
   'max_tokens': 2000,
   'GPU_I

In [17]:
model_name="hatakeyama-llm-team/Tanuki-8B-Instruct"
model_name="microsoft/Phi-3-medium-128k-instruct"
model_name="Rakuten/RakutenAI-7B-chat"
model_name="karakuri-ai/karakuri-lm-8x7b-chat-v0.1"

question="純粋理性批判はたぬきに理解できますか?"
res=ask_llm_prompt(client_dict,model_name,question)
res

'申し訳ありませんが、前提条件を満たしていないと思われます。純粋理性批判は哲学の書物であり、たぬきに理解させるのは困難が伴うと思われます。'

In [39]:


template1="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
template2="\n\n### 応答:\n"
prompt=template1+question+template2
completion = client.completions.create(model=model,
                                      prompt=prompt)
                                      
completion

Completion(id='cmpl-0d2ade79a5df4a51a11612391f123c2b', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' はい、元気です。あなたはどうですか？何かお困りのこと', stop_reason=None)], created=1717579730, model='hatakeyama-llm-team/Tanuki-8B-Instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=55, total_tokens=71))

In [43]:
import random
import time
import datetime
from src.GeneralBot import GeneralBot
from src.TanukiBot import TanukiBot
from src.Client import Client

# %%

with open("env/url.txt") as f:
    url = f.read().strip()

# apiクライアントとchatbotを起動
client = Client(url)




# %%
model_list = list(client_dict.keys())


# %%
while True:
    row_id, question, inst = client.get_unanswered_question()
    if question == "":
        print("no question to answer")
        time.sleep(10)
        continue

    print(question)
    model_name_A, model_name_B = random.sample(model_list, 2)
    responseA=ask_llm(client_dict,model_name_A,question)
    responseB=ask_llm(client_dict,model_name_B,question)
    meta1 = client_dict[model_name_A]["model"]
    meta2 = client_dict[model_name_B]["model"]
    meta3 = datetime.datetime.now().isoformat()

    client.answer(row_id, responseA, responseB, metainfo1=meta1,
                  metainfo2=meta2, metainfo3=meta3)



あなたは優秀な編集者です。次の文章を校正して下さい。「イタレーション速度が当初想定していたよりも進みが遅い」
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
no question to answer
今は何時ですか
